## Using Foursquare Map and Segmentation for venues in toronto 

In [65]:
import pandas as pd 
import requests
import numpy as np
from sklearn.cluster import KMeans
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors


In [3]:
df=pd.read_csv('Toronto_LATLONG_cleaned.csv')
df.head(2)

,Postalcode,Latitude,Longitude,Borough,Neighborhoods
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"


In [4]:
CLIENT_ID = '2I21WCHSLPW2VDRCBAWPD4TGGLJZX2WK1PFZGMJNMCDNDZN3' # your Foursquare ID
CLIENT_SECRET = 'GSHJQ0VRKCY1JLZKOG4A0DECW5T2QAV1ZJSIICDZOR1KSL3Z' # your Foursquare Secret
VERSION = '20201221'

In [5]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
toronto_venues = getNearbyVenues(names=df['Neighborhoods'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [18]:
toronto_venues.head(3)
toronto_venues.shape

(2124, 7)

In [15]:
print ("number of unique venues category "+str(len(toronto_venues['Venue Category'].unique())))

number of unique venues category 271


In [24]:
#let's do one hot encoding for venues category 
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(2124, 271)

In [26]:
#Now, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped.shape


(96, 271)

### Return most common venues for each category using sort descending 

In [27]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Pool,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Ice Cream Shop,Fried Chicken Joint,Bridal Shop
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Thai Restaurant,Liquor Store


### Clustering neighborhoods


In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

### Adding kmean labels to sorted venues

In [54]:

# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



# check the last columns!
toronto_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,KmeanLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant,4,4,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Escape Room
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping,1,1,Construction & Landscaping,Bar,Yoga Studio,Donut Shop,Diner
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,1,1,Construction & Landscaping,Bar,Yoga Studio,Donut Shop,Diner
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank,1,1,Electronics Store,Intersection,Breakfast Spot,Restaurant,Rental Car Location
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,1,1,Electronics Store,Intersection,Breakfast Spot,Restaurant,Rental Car Location


## Visualize the clusters in map 

In [69]:
address = 'Toronto, CA'

latitude = 43.653963
longitude = -79.387207


In [70]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Venue Latitude'], toronto_merged['Venue Longitude'], toronto_merged['Neighborhood'], toronto_merged['KmeanLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### printing the five clusters 

In [77]:
toronto_merged.loc[toronto_merged['KmeanLabels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,KmeanLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,43.770992,-79.221156,Coffee Shop,0,0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Pharmacy,Ethiopian Restaurant
12,43.770992,-79.223078,Coffee Shop,0,0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Pharmacy,Ethiopian Restaurant
13,43.770992,-79.214502,Korean BBQ Restaurant,0,0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Pharmacy,Ethiopian Restaurant
14,43.770992,-79.218210,Soccer Field,0,0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Pharmacy,Ethiopian Restaurant
15,43.770992,-79.222500,Pharmacy,0,0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Pharmacy,Ethiopian Restaurant
...,...,...,...,...,...,...,...,...,...,...
2103,43.696319,-79.531471,Sandwich Place,0,0,Pizza Place,Chinese Restaurant,Intersection,Coffee Shop,Sandwich Place
2104,43.696319,-79.531570,Pizza Place,0,0,Pizza Place,Chinese Restaurant,Intersection,Coffee Shop,Sandwich Place
2105,43.696319,-79.531698,Middle Eastern Restaurant,0,0,Pizza Place,Chinese Restaurant,Intersection,Coffee Shop,Sandwich Place
2106,43.696319,-79.534408,Intersection,0,0,Pizza Place,Chinese Restaurant,Intersection,Coffee Shop,Sandwich Place


In [78]:
toronto_merged.loc[toronto_merged['KmeanLabels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,KmeanLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,43.784535,-79.163742,Construction & Landscaping,1,1,Construction & Landscaping,Bar,Yoga Studio,Donut Shop,Diner
2,43.784535,-79.163085,Bar,1,1,Construction & Landscaping,Bar,Yoga Studio,Donut Shop,Diner
3,43.763573,-79.191151,Bank,1,1,Electronics Store,Intersection,Breakfast Spot,Restaurant,Rental Car Location
4,43.763573,-79.191537,Electronics Store,1,1,Electronics Store,Intersection,Breakfast Spot,Restaurant,Rental Car Location
5,43.763573,-79.191275,Restaurant,1,1,Electronics Store,Intersection,Breakfast Spot,Restaurant,Rental Car Location
...,...,...,...,...,...,...,...,...,...,...
2119,43.739416,-79.584230,Fast Food Restaurant,1,1,Grocery Store,Pharmacy,Pizza Place,Beer Store,Fast Food Restaurant
2120,43.706748,-79.589943,Rental Car Location,1,1,Bar,Rental Car Location,Drugstore,Truck Stop,Yoga Studio
2121,43.706748,-79.589252,Bar,1,1,Bar,Rental Car Location,Drugstore,Truck Stop,Yoga Studio
2122,43.706748,-79.598725,Drugstore,1,1,Bar,Rental Car Location,Drugstore,Truck Stop,Yoga Studio


In [79]:
toronto_merged.loc[toronto_merged['KmeanLabels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,KmeanLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
78,43.815252,-79.281716,Intersection,2,2,Park,Intersection,Playground,Dog Run,Dessert Shop
79,43.815252,-79.289773,Park,2,2,Park,Intersection,Playground,Dog Run,Dessert Shop
80,43.815252,-79.289867,Playground,2,2,Park,Intersection,Playground,Dog Run,Dessert Shop
170,43.789053,-79.405494,Park,2,2,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant
204,43.752758,-79.401393,Convenience Store,2,2,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner
205,43.752758,-79.399717,Park,2,2,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner
211,43.753259,-79.332140,Park,2,2,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
212,43.753259,-79.333114,Food & Drink Shop,2,2,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
360,43.685347,-79.335007,Park,2,2,Convenience Store,Park,Intersection,Doner Restaurant,Dim Sum Restaurant
361,43.685347,-79.340100,Intersection,2,2,Convenience Store,Park,Intersection,Doner Restaurant,Dim Sum Restaurant


In [80]:
toronto_merged.loc[toronto_merged['KmeanLabels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,KmeanLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2073,43.636258,-79.496266,Baseball Field,3,3,Baseball Field,Donut Shop,Dim Sum Restaurant,Diner,Discount Store
2096,43.724766,-79.532854,Baseball Field,3,3,Baseball Field,Furniture / Home Store,Yoga Studio,Dim Sum Restaurant,Diner
2097,43.724766,-79.529923,Furniture / Home Store,3,3,Baseball Field,Furniture / Home Store,Yoga Studio,Dim Sum Restaurant,Diner


In [81]:
toronto_merged.loc[toronto_merged['KmeanLabels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,KmeanLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,43.806686,-79.199056,Fast Food Restaurant,4,4,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Escape Room
1906,43.691116,-79.474050,Sandwich Place,4,4,Fast Food Restaurant,Sandwich Place,Discount Store,Yoga Studio,Dog Run
1907,43.691116,-79.474667,Discount Store,4,4,Fast Food Restaurant,Sandwich Place,Discount Store,Yoga Studio,Dog Run
1908,43.691116,-79.479982,Fast Food Restaurant,4,4,Fast Food Restaurant,Sandwich Place,Discount Store,Yoga Studio,Dog Run
1909,43.691116,-79.480948,Fast Food Restaurant,4,4,Fast Food Restaurant,Sandwich Place,Discount Store,Yoga Studio,Dog Run


In [83]:
#saving pdf_for future
toronto_merged.to_csv('Toronto_final.csv',index=False)